In [66]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np

from numpy import float32
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch import optim

In [67]:
train_path = './digit-recognizer/train.csv'
train_data = pd.read_csv(train_path, dtype=np.float32)

test_path = './digit-recognizer/test.csv'
test_data = pd.read_csv(test_path)

In [68]:

X_train = train_data.drop('label', axis=1).to_numpy().reshape(-1, 28, 28).astype(torch.float32) / 255
X_test = test_data.to_numpy().reshape(-1, 28, 28).astype(torch.float32) / 255

Y_train = train_data['label']

#plt.imshow(X_train[1].reshape(28, 28))

TypeError: Cannot interpret 'torch.float32' as a data type

In [ ]:
train_tensor = torch.Tensor(X_train)
test_tensor = torch.Tensor(X_test)
label_tensor = torch.Tensor(Y_train)

train_tensor = TensorDataset(train_tensor, label_tensor)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
batch_size = 64
trainloader = DataLoader(train_tensor, batch_size=batch_size)
testloader = DataLoader(test_tensor, batch_size=batch_size)

In [ ]:
for image, label in trainloader:
    print(image.shape)

torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])
torch.Size([64, 28, 28])


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.convNet = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.linearNet = nn.Sequential(
            nn.Linear(7*7*128, 256),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Dropout(0.2),
            nn.ReLU(),
        )
        
    def forward(self, x):
        x = self.convNet(x)
        x = x.view(x.shape[0], -1)
        x = self.linearNet(x)
        return x
        
model = Network()
model.to(device)

print(model)

Using cuda
Network(
  (convNet): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linearNet): Sequential(
    (0): Linear(in_features=6272, out_features=256, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): Dropout(p=0.2, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=64, out_featu

In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [ ]:
for batch, (image, label) in enumerate(trainloader):
    image, label = image.to(device), label.to(device)
    image = image.resize(batch_size, 1, 28, 28)

    optimizer.zero_grad()
    output = model(image)
    loss_out = loss(output, label)
    loss_out.backward()
    optimizer.step()
    
    print(loss.item())
    break
    # if(batch % 100 == 0):
    #   print(f"Loss: {loss.item()}")

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'